In [56]:
import pprint
from typing import List
from dataclasses import dataclass
import pathlib
import collections

@dataclass
class Module:
    type: str
    name: str
    connections: List[str]

def parse_line(line):
    a, b = line.split('->')
    a, b = a.strip(), b.strip()
    type, name = (None, None)
    match a[0]:
        case '%':
            type, name = 'flipflop', a[1:]
        case '&':
            type, name = 'conjunction', a[1:]
        case _ if a == 'broadcaster':
            type, name = 'broadcaster', 'broadcaster'
        case _:
            raise Exception(f'Unexpected first char {a[0]}')

    connections = [x.strip() for x in b.split(',')]
    return Module(
        type = type,
        name = name,
        connections = connections,
    )

def is_tree(start, graph, visited = None):
    """ Copy pasted from d19/explore.py """

    visited = visited or []

    if start in visited:
        print(f'found loop involving {start}', visited)
        return False

    child_nodes = graph[start]
    return all(
        is_tree(cn, graph, visited + [start])
        for cn in child_nodes
    )

import os
print(os.getcwd())

path = pathlib.Path.cwd()
while path.name != "advent-of-code-2023" and path != path.parent:
    path = path.parent

with (path / pathlib.Path('src/d20/input')).open() as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]

modules = [parse_line(line) for line in lines]

_dst_module_names = [child for mod in modules for child in mod.connections]
_name_graph = {
        mod.name: mod.connections
        for mod in modules
}
blackhole_module_names = list(set(name for name in _dst_module_names if name not in _name_graph.keys()))
for bhole_name in blackhole_module_names:
    modules.append(Module(name = bhole_name, type = 'blackhole', connections = []))
# for name in blackhole_module_names
# print('blackhole', blackhole_module_names)
# pprint.pprint(collections.OrderedDict(sorted([(k, v) for k, v in name_graph.items()])))


#print(is_tree('broadcaster', graph))

/Users/evanthomas/github.com/ethomas2/advent-of-code-2023/src/d20


In [57]:
name_to_mod = {mod.name: mod for mod in modules}

module_graph = {
    mod.name: [name_to_mod[child_name] for child_name in mod.connections] 
    for mod in modules
}            

In [60]:
from pyvis.network import Network
net = Network(notebook=True, directed=True)
# net.toggle_physics(True)

# Have to add all nodes first
for mod in modules:
    color = {
        'flipflop': 'blue',
        'conjunction': 'green',
        'broadcaster': 'red',
        'blackhole': 'black',
    }[mod.type]
    net.add_node(mod.name, label=mod.name, color = color)

for (src_name, dst_modules) in module_graph.items():
    # print('arr', dst_modules)
    for dst_mod in dst_modules:
        # print('single', dst_mod)
        net.add_edge(src_name, dst_mod.name)

#net.show_buttons(filter_=['physics'])
net.show("example.html")


example.html
